# 201601615 이의찬

# 1. 지정 주제

In [162]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By # find_element 를 할 때, 속성값 관리
from urllib.request import urlopen
from tqdm.notebook import tqdm
import time

In [160]:
driver_path = "chromedriver.exe" # 경로명: 현재 폴더로 되어있음
movie_url = r"https://movie.daum.net/moviedb/grade?movieId=62730" # 다음 영화: 인터스텔라

### selenium 드라이버

In [168]:
driver = webdriver.Chrome(driver_path)
driver.implicitly_wait(3)
driver.get(movie_url)
time.sleep(2)

C:\Users\outda\AppData\Local\Temp\ipykernel_11976\2398086757.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


### 평점 더보기 버튼, 버튼 누를 횟수

In [169]:
more_button = '//*[@id="alex-area"]/div/div/div/div[3]/div[1]/button' # 더보기 버튼
user_comments = [] # 유저 댓글 정보 리스트

user_count = driver.find_element(
    By.CLASS_NAME, "txt_netizen"
).text.strip("()명") # 평점을 남긴 네티즌 수

more_count = int(user_count) // 10 # more 버튼 누르는 총 횟수
more_count = 20 # more 버튼 test 개수

for i in tqdm(range(more_count)):
    driver.find_element("xpath", more_button).click()
    time.sleep(2)

  0%|          | 0/10 [00:00<?, ?it/s]

### BeautifulSoup으로 데이터 수집

#### driver.page_source 사용 이유

```python
html = urlopen(movie_url)
soup = BeautifulSoup(html.read(), "html.parser")
```

`BeautifulSoup`을 사용할 경우 모든 태그를 가져오지 않는다.(::after로 숨겨져 있음) <br>
`driver.page_source`를 사용하여 현재 렌더링 된 페이지의 모든 `Elements`를 모두 가져올 수 있다. <br>


In [180]:
html = driver.page_source # selenium으로 작업한 url 정보를 html로 가져옴
soup = BeautifulSoup(html, "html.parser")

user_reviews = [] # 유저 별점과 댓글, 타입: list<tuple<별점:int, 리뷰:str>>

reviews = soup.find_all("div", {"class": "cmt_info"})

for review in tqdm(reviews):
    comment = review.find("p", {"class": "desc_txt font_size_"}) # 댓글
    
    if not comment:
        # 댓글을 적지 않고 평점만 줬다면 무시
        continue
        
    rating = int(review.find("div").text) # 평점: 첫번째 div
    comment = comment.text.strip() # 댓글
    
    user_reviews.append((rating, comment))

  0%|          | 0/310 [00:00<?, ?it/s]

In [181]:
user_reviews

[(10,
  '30번째 다시 보는 중인데\n새로운 것을 또 찾았다.\n이 영화는 매 장면마다 감독이 뭔가를 심어놨어...\n\n미래의 아빠가 과거의 딸에게 보내는 이진코드 신호가\n연속적인 시간이 아니었다.\n\n이진코드로 나사위치 알려준거...\n돈고! 스뗴이! 라고 외치는거 다음에\n딸에게 모든 지식 알려주고 나서\n딸이 사는 세계가 안전해진것을 알게되고나서\n과거의 자신에게 나사 위치 알려준거다\n\n아빠는\n결국 자신이 딸 곁에 남는것보다 (스떼이! 스뗴이!)\n딸이 살아갈 세상을 지키는것 (나사 위치 알려줌)\n을 선택 한 것이었다.\n\n이런거 영화 평론에도 없는데\n내가 몇년간 계속 돌려보면서 하나씩 찾아냄'),
 (9,
  '인류의 미래, 꿈, 과학, 우주, 인류애, 가족 등등 인간의 삶을 상상력으로 적합해 어떻게 이런 영화를 만들어 낼수 있는지 몇번을 봐도 놀라운 명작'),
 (9,
  '오ㅏ~~ 뒷부분 미래의 아빠가 과거의 딸을 책장 너머로 보는 장면부터 소름이 쫙~ 쫙~~ 몇번이 끼쳤는지.. 👍\n같이 우주로 떠났던 박사중 한명이 종이에 x 자 두군데 그리고 접어서 볼펜으로 관통시키며 설명했던 그 원리 아니였나? (잘은 모르지만 난 그렇게 이해함 ㅋㅋㅋ)\n늙은 딸을 만나는 장면에선 눈물이~~~~ 😭\n감동과 보는 재미 인류애와 현재 우리에게 다가올 과제등을 두루 다 느끼게 해주는 영화\n다들 연기도 좋았고 암튼 다 너무 좋았음 최고~!! 👍'),
 (10,
  '어렵고 복잡한 수학적, 과학적 아이디어를 관객이 따라가기 쉽게, 그리고 현실처럼 생생하게 느끼면서 인간적인 면까지 조명하여 만든 할리우드 영화.  신이나 초자연적인 메시아가 아닌 우리자신이 스스로를 구하고, 문명을 진화시켜야한다는 주제가 있다고 생각했다. 그것을 가능케 하는것은 사랑의 힘이다'),
 (10,
  '삶의 주인공은 자신이라는 것을 "블랙홀.시간.공간.그리움.다차원우주.그리고 내 삶을 찾아 떠나는....불멸의 수수께기 "블랙홀"'),
 (8,
  '4차원 5차원의 세계